In [1]:
!pip install asgl

  Preparing metadata (setup.py) ... done
  Created wheel for asgl: filename=asgl-2.1.4-py3-none-any.whl size=32845 sha256=d0932ef418f7001725a0038e92e88c3e7a98f0682b10e471de45509240e3f376
  Stored in directory: /root/.cache/pip/wheels/19/43/4f/a84b6af3395f9c157b503207f768533f29615b2fee20ce08a0
Successfully built asgl


In [2]:
import numpy as np
import pandas as pd

import time
import warnings
import pickle
import optuna

from asgl import Regressor
import random
random.seed(42)
np.random.seed(42)
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/home/a/桌面/Simulation code and data/data/time_series_15min_cleaned.csv')#res_new.csv

a = np.array(df.values[:, 2])#fujian不适用
a = np.expand_dims(a, axis=1)#fujian不适用

def split_data(data, horizon, window):
    n, m = data.shape
    X = np.zeros((n - window - horizon, window))
    Y = np.zeros((n - window - horizon, 1))

    for i in range(n - window - horizon):
        start = i
        end = start + window
        X[i, :] = data[start:end, 0]
        Y[i] = data[end + horizon - 1, 0]

    return X, Y
#fujian dataset
#def split_data(data, horizon, window):
#    n, m = data.shape
#    X = np.zeros((n - window - horizon, window))
#    Y = np.zeros((n - window - horizon, 1))
#    EX = np.zeros((n - window - horizon, 2))
#    for i in range(n - window - horizon):
#        start = i
#        end = start + window
#        X[i, :] = data[start:end, 0]
#        Y[i] = data[end + horizon - 1, 0]
#        EX[i, :] = data[end + horizon - 1, 5:6]
#    return X, Y, EX

X, Y = split_data(a, 1, 96)
n1 = X.shape[0]
#X_arg = np.hstack((X, EX))#fujian dataset
train_X, train_Y = X[:round(0.8 * n1)], Y[:round(0.8 * n1)]
val_X, val_Y = X[round(0.8 * n1):round(0.9 * n1)], Y[round(0.8 * n1):round(0.9 * n1)]
test_X, test_Y = X[round(0.9 * n1):], Y[round(0.9 * n1):]

tau_values = [round(i, 2) for i in np.arange(0.05, 1.0, 0.05)]

In [4]:
def pinball_loss(y_true, y_pred, tau):
    """
    计算单个分位数的pinball loss
    """
    errors = y_true - y_pred
    loss = np.where(errors >= 0, tau * errors, (tau - 1) * errors)
    return np.mean(loss)


def multi_quantile_pinball_loss(y_true, y_pred_all, tau_values):
    """
    计算多个分位数的平均pinball loss
    """
    total_loss = 0
    n_quantiles = len(tau_values)

    for i, tau in enumerate(tau_values):
        total_loss += pinball_loss(y_true, y_pred_all[:, i], tau)

    return total_loss / n_quantiles

In [5]:
def objective(trial):
    """
    Optuna优化目标函数
    """
    # 定义lambda搜索范围（标准化后可以调整搜索范围）
    lambda_val = trial.suggest_int('lambda', 1, 10)

    # 存储每个分位数的验证集预测
    predictions_val = []

    start_time = time.time()

    for tau in tau_values:
        # 创建模型
        model = Regressor(model='qr', penalization='lasso', quantile=tau, lambda1=lambda_val)

        # 训练模型（使用标准化后的X）
        model.fit(train_X, train_Y)

        # 在验证集上预测（使用标准化后的X）
        y_pred_val = model.predict(val_X).reshape(-1, 1)
        predictions_val.append(y_pred_val)

    # 组合所有分位数的预测
    predictions_val_matrix = np.hstack(predictions_val)

    # 计算平均pinball loss
    current_loss = multi_quantile_pinball_loss(val_Y, predictions_val_matrix, tau_values)

    trial_time = time.time() - start_time
    trial.set_user_attr('time', trial_time)

    return current_loss

In [6]:
def optimize_lambda(n_trials=10):
    """
    使用Optuna优化lambda参数
    """
    print(f"开始Optuna优化，共{n_trials}次试验...")

    # 创建Optuna study
    study = optuna.create_study(
        direction='minimize',
        sampler=optuna.samplers.TPESampler(seed=42),
    )

    # 执行优化
    start_time = time.time()
    study.optimize(objective, n_trials=n_trials, show_progress_bar=True)
    total_time = time.time() - start_time

    print(f"\n优化完成，总耗时: {total_time:.3f}秒")
    print(f"最佳lambda: {study.best_params['lambda']:.6f}")
    print(f"最佳验证损失: {study.best_value:.6f}")

    # 保存优化结果
    results = {
        'best_lambda': study.best_params['lambda'],
        'best_loss': study.best_value,
        'study': study,
        'total_time': total_time,
    }

    with open("optuna_lasso_results.pkl", "wb") as f:
        pickle.dump(results, f)

    print("optuna_lasso_results.pkl")

    return results

In [ ]:
optuna_results = optimize_lambda(n_trials=10)
best_lambda = optuna_results['best_lambda']

best_params = {
    'lambda': optuna_results['best_lambda'],
}

import json
with open('lassobest_params.json', 'w') as f:
    json.dump(best_params, f)

print("超参数已保存到 best_params.json")

[I 2026-01-28 14:59:40,304] A new study created in memory with name: no-name-164f413b-b366-496a-85b3-0b30fbf594d9


开始Optuna优化，共10次试验...


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2026-01-28 15:15:56,303] Trial 0 finished with value: 894.1733210314029 and parameters: {'lambda': 4}. Best is trial 0 with value: 894.1733210314029.
[I 2026-01-28 15:31:58,103] Trial 1 finished with value: 892.6252893851049 and parameters: {'lambda': 10}. Best is trial 1 with value: 892.6252893851049.


In [5]:
import json

with open('/kaggle/input/offshoreforecasting/lassobest_params.json', 'r') as f:
    best_params = json.load(f)

best_lambda = best_params['lambda']
print(f"已加载超参数: lambda = {best_lambda}")

已加载超参数: lambda = 10


In [6]:
# 使用最佳lambda在测试集上评估
def evaluate_on_test_set(lambda_val):

    print(f"\n在测试集上评估，lambda={lambda_val:.6f}")

    test_predictions = []

    start_time = time.time()

    for tau in tau_values:
        model = Regressor(model='qr', penalization='lasso', quantile=tau, lambda1=lambda_val)

        model.fit(train_X, train_Y)
        y_pred_test = model.predict(test_X).reshape(-1, 1)
        test_predictions.append(y_pred_test)

    test_predictions_matrix = np.hstack(test_predictions)

    # 计算测试集上的平均pinball loss
    test_loss = multi_quantile_pinball_loss(test_Y, test_predictions_matrix, tau_values)

    eval_time = time.time() - start_time

    print(f"\n{'=' * 60}")
    print(f"测试集评估结果:")
    print(f"使用的lambda: {lambda_val:.6f}")
    print(f"测试集平均pinball loss: {test_loss:.3f}")
    print(f"评估耗时: {eval_time:.3f}秒")
    print(f"{'=' * 60}")

    return test_loss, test_predictions_matrix

In [7]:
test_loss, test_predictions_inv = evaluate_on_test_set(best_lambda)


在测试集上评估，lambda=10.000000

测试集评估结果:
使用的lambda: 10.000000
测试集平均pinball loss: 919.047
评估耗时: 1078.401秒


In [9]:
columns = [f'q{int(tau * 100)}' for tau in tau_values]
all_pred_df = pd.DataFrame(test_predictions_inv, columns=columns)
all_pred_df.to_excel('lassopred.xlsx', index=True)